In [ ]:
import tensorflow as tf
import json
# import keras
# from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf
from tensorflow.python.keras.layers import *
from tensorflow.python import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
blog_posts_data_dir = "/content/drive/MyDrive/datasets/"
blog_posts_data_dir = "/content/drive/MyDrive/datasets/BT-AP-19 Corpus/all/"
train_file_name = "train.json"
test_file_name = "test.json"

# Load data
with open(blog_posts_data_dir + train_file_name) as r:
    training_set = json.load(r)
training_set[0]

{'age': '50', 'gender': 'female', 'post': 'Umar Murtaza: Head of SM Operation'}

In [ ]:
EXAMINE = 21
SEED = 22
np.random.seed(SEED)

In [ ]:
def get_gender_as_num(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    if age < 18:
        # 13 - 17
        return [1, 0, 0, 0]
    elif age < 28:
        # 23 - 27
        return [0, 1, 0, 0]
    elif age < 49:
        # 33 - 48
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1]

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    age = int(age)
    if age == 18:
        # 13 - 17
        return [1, 0, 0, 0]
    elif age == 25:
        # 23 - 27
        return [0, 1, 0, 0]
    elif age == 35:
        # 33 - 48
        return [0, 0, 1, 0]
    elif age == 50:
        # 33 - 48
        return [0, 0, 0, 1]
    else:
        return [0, 0, 0, 0]

In [ ]:
raw_posts = [instance["post"] for instance in training_set]
len(raw_posts)

33637

In [ ]:
# TODO add stop word filtering 
median_words_per_sample = np.median([len(instance["post"]) for instance in training_set])

# Map each word to a unique int value
MAX_WORD_COUNT = 20000
tokenizer = keras.preprocessing.text.Tokenizer(num_words = MAX_WORD_COUNT)
posts = [instance["post"] for instance in training_set]
tokenizer.fit_on_texts(posts)
word_index = dict(list(tokenizer.word_index.items())[:20000])
sequences = tokenizer.texts_to_sequences(posts)
median_words_per_tokenized_sample = np.median([len(post) for post in sequences])
data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(training_set):
    instance["post"] = data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
EMBEDDING_DIM = 50

glove_path = "/content/drive/MyDrive/datasets/"
glove_dict = {}
with open(glove_path + "glove.6B.50d.txt") as f:
    for line in f:
        line_values = line.split(" ")
        word = line_values[0]
        embedding_coefficients = np.asarray(line_values[1 : ], dtype = "float32")
        glove_dict[word] = embedding_coefficients

glove_weights = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    glove_vector = glove_dict.get(word)
    if glove_vector is not None:
        glove_weights[i] = glove_vector

In [ ]:
model_1 = keras.Sequential()
model_1.add(keras.layers.Embedding(len(word_index) + 1, EMBEDDING_DIM, weights = [glove_weights],
                                    input_length = int(median_words_per_tokenized_sample), trainable = True))


# Add hidden layers 
for i in range(0, 3):
    # Add a bidirectional lstm layer
    model_1.add(Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.2)))
    # Add a dropout layer after each lstm layer
    model_1.add(Dropout(0.5))
model_1.add(Bidirectional(LSTM(32, recurrent_dropout=0.2)))
model_1.add(Dropout(0.5))
# Add the fully connected layer with 256 nurons and relu activation
model_1.add(Dense(256, activation='relu'))
# Add the output layer with softmax activation since we have 2 classes
model_1.add(Dense(3, activation='softmax'))

In [ ]:
model_1.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 50)             1000050   
_________________________________________________________________
bidirectional (Bidirectional (None, 9, 64)             21248     
_________________________________________________________________
dropout (Dropout)            (None, 9, 64)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 9, 64)             24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 9, 64)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 9, 64)             24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 9, 64)             0

In [ ]:
def create_model(type=True):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(len(word_index) + 1, EMBEDDING_DIM, weights = [glove_weights],
                                        input_length = int(median_words_per_tokenized_sample), trainable = True))


    # Add hidden layers 
    for i in range(0, 3):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(32, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(32, recurrent_dropout=0.2)))
    model.add(Dropout(0.5))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(256, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    if type: 
        model.add(Dense(4, activation='softmax')) 
    else:
        model.add(Dense(1, activation='sigmoid')) 

    return model

In [ ]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model = create_model()
    model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])
    model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.102.178.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.178.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 9, 50)             1000050   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 9, 64)             21248     
_________________________________________________________________
dropout_4 (Dropout)          (None, 9, 64)             0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 9, 64)             24832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 9, 64)             0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 9, 64)             24832     
_________________________________________________________________
dropout_6 (Dropout)          (None, 9, 64)            

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
ages_train = np.array([instance["age"] for instance in training_set])

In [ ]:
history_1 = model.fit(posts_train, ages_train, epochs = 10, batch_size = 50, validation_split = 0.2)

Epoch 1/10
539/539 [==============================] - 30s 41ms/step - loss: 0.7814 - accuracy: 0.6745 - val_loss: 1.2676 - val_accuracy: 0.5059
Epoch 2/10
539/539 [==============================] - 13s 24ms/step - loss: 0.6790 - accuracy: 0.7202 - val_loss: 1.3191 - val_accuracy: 0.5100
Epoch 3/10
539/539 [==============================] - 13s 24ms/step - loss: 0.6151 - accuracy: 0.7464 - val_loss: 1.3314 - val_accuracy: 0.5043
Epoch 4/10
539/539 [==============================] - 13s 24ms/step - loss: 0.5763 - accuracy: 0.7619 - val_loss: 1.5357 - val_accuracy: 0.5165
Epoch 5/10
539/539 [==============================] - 13s 24ms/step - loss: 0.5370 - accuracy: 0.7738 - val_loss: 1.5966 - val_accuracy: 0.5123
Epoch 6/10
539/539 [==============================] - 13s 24ms/step - loss: 0.5149 - accuracy: 0.7837 - val_loss: 1.6219 - val_accuracy: 0.5062
Epoch 7/10
539/539 [==============================] - 13s 24ms/step - loss: 0.4838 - accuracy: 0.7954 - val_loss: 1.7493 - val_accuracy:

In [ ]:
# Load data
with open(blog_posts_data_dir + test_file_name) as r:
    test_set = json.load(r)

In [ ]:
test_posts = [instance["post"] for instance in test_set]
test_sequences = tokenizer.texts_to_sequences(test_posts)
test_post_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(test_set):
    instance["post"] = test_post_data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
ages_test = np.array([instance["age"] for instance in test_set])

In [ ]:
model.evaluate(posts_test, ages_test)

263/263 [==============================] - 5s 17ms/step - loss: 1.8763 - accuracy: 0.4872


[1.8762813806533813, 0.4871581494808197]

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict(posts_test)
print(classification_report(np.argmax(ages_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.54      0.66      0.59      2698
           1       0.52      0.40      0.45      2647
           2       0.45      0.43      0.44      2309
           3       0.33      0.36      0.34       756

    accuracy                           0.49      8410
   macro avg       0.46      0.46      0.46      8410
weighted avg       0.49      0.49      0.48      8410



## Gender

In [ ]:
def create_model_gender():
    model = keras.Sequential()
    model.add(keras.layers.Embedding(len(word_index) + 1, EMBEDDING_DIM, weights = [glove_weights],
                                        input_length = int(median_words_per_tokenized_sample), trainable = True))


    # Add hidden layers 
    for i in range(0, 3):
        # Add a bidirectional lstm layer
        model.add(Bidirectional(LSTM(64, return_sequences=True, recurrent_dropout=0.2)))
        # Add a dropout layer after each lstm layer
        model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(64, recurrent_dropout=0.2)))
    model.add(Dropout(0.5))
    # Add the fully connected layer with 256 nurons and relu activation
    model.add(Dense(10, activation='relu'))
    # Add the output layer with softmax activation since we have 2 classes
    model.add(Dense(1, activation='sigmoid')) 

    return model

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
genders_train = np.array([instance["gender"] for instance in training_set])

In [ ]:
import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    model_gender = create_model_gender()
    model_gender.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    model_gender.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.102.178.10:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.102.178.10:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 9, 50)             1000050   
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 9, 128)            58880     
_________________________________________________________________
dropout_12 (Dropout)         (None, 9, 128)            0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 9, 128)            98816     
_________________________________________________________________
dropout_13 (Dropout)         (None, 9, 128)            0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 9, 128)            98816     
_________________________________________________________________
dropout_14 (Dropout)         (None, 9, 128)           

In [ ]:
history_gender = model_gender.fit(posts_train, genders_train, epochs = 10, batch_size = 50, validation_split = 0.2)

Epoch 1/10
539/539 [==============================] - 52s 46ms/step - loss: 0.6011 - accuracy: 0.6922 - val_loss: 0.5499 - val_accuracy: 0.7250
Epoch 2/10
539/539 [==============================] - 13s 25ms/step - loss: 0.5154 - accuracy: 0.7541 - val_loss: 0.5169 - val_accuracy: 0.7493
Epoch 3/10
539/539 [==============================] - 13s 24ms/step - loss: 0.4289 - accuracy: 0.8043 - val_loss: 0.5173 - val_accuracy: 0.7516
Epoch 4/10
539/539 [==============================] - 13s 25ms/step - loss: 0.3399 - accuracy: 0.8481 - val_loss: 0.5481 - val_accuracy: 0.7632
Epoch 5/10
539/539 [==============================] - 13s 25ms/step - loss: 0.2822 - accuracy: 0.8750 - val_loss: 0.5361 - val_accuracy: 0.7579
Epoch 6/10
539/539 [==============================] - 13s 25ms/step - loss: 0.2510 - accuracy: 0.8919 - val_loss: 0.6584 - val_accuracy: 0.7567
Epoch 7/10
539/539 [==============================] - 13s 24ms/step - loss: 0.2175 - accuracy: 0.9023 - val_loss: 0.7048 - val_accuracy:

In [ ]:
# Load data
with open(blog_posts_data_dir + test_file_name) as r:
    test_set = json.load(r)

In [ ]:
test_posts = [instance["post"] for instance in test_set]
test_sequences = tokenizer.texts_to_sequences(test_posts)
test_post_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(test_set):
    instance["post"] = test_post_data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
genders_test = np.array([instance["gender"] for instance in test_set])

In [ ]:
model_gender.evaluate(posts_test, genders_test)

263/263 [==============================] - 5s 18ms/step - loss: 0.9911 - accuracy: 0.7512


[0.9910658001899719, 0.7512485384941101]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
pred = model_gender.predict(posts_test)

In [ ]:
print(classification_report(genders_test, (pred > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.61      0.50      0.55      2565
           1       0.80      0.86      0.83      5845

    accuracy                           0.75      8410
   macro avg       0.71      0.68      0.69      8410
weighted avg       0.74      0.75      0.74      8410

